In [1]:
# Cell 1: Basic imports & session (you already have this, but good habit)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when

spark = SparkSession.builder.appName("PySparkBasics").getOrCreate()

In [2]:
# Cell 2 - Corrected data creation
data = [
    ("Alice", 28, "Engineer", 85000, "Austin"),
    ("Bob", 35, "Manager", 120000, "Dallas"),
    ("Charlie", 22, "Intern", 45000, "Houston"),
    ("Dana", 40, "Director", 180000, "Plano"),
    ("Eve", None, "Analyst", 95000, "Frisco")
]

columns = ["Name", "Age", "Job", "Salary", "City"]
df = spark.createDataFrame(data, columns)
df.show()

+-------+----+--------+------+-------+
|   Name| Age|     Job|Salary|   City|
+-------+----+--------+------+-------+
|  Alice|  28|Engineer| 85000| Austin|
|    Bob|  35| Manager|120000| Dallas|
|Charlie|  22|  Intern| 45000|Houston|
|   Dana|  40|Director|180000|  Plano|
|    Eve|NULL| Analyst| 95000| Frisco|
+-------+----+--------+------+-------+



In [3]:
# Cell 3 - Corrected chain (missing . after withColumn for Bonus)
df_clean = df \
    .withColumn("Senior", when(col("Age") >= 30, "Yes").otherwise("No")) \
    .withColumn("Bonus", col("Salary") * 0.10) \
    .na.fill({"Age": 25})

df_clean.show()

+-------+---+--------+------+-------+------+-------+
|   Name|Age|     Job|Salary|   City|Senior|  Bonus|
+-------+---+--------+------+-------+------+-------+
|  Alice| 28|Engineer| 85000| Austin|    No| 8500.0|
|    Bob| 35| Manager|120000| Dallas|   Yes|12000.0|
|Charlie| 22|  Intern| 45000|Houston|    No| 4500.0|
|   Dana| 40|Director|180000|  Plano|   Yes|18000.0|
|    Eve| 25| Analyst| 95000| Frisco|    No| 9500.0|
+-------+---+--------+------+-------+------+-------+



In [4]:
# Cell 4 - Fixed agg & rename syntax
df_clean.groupBy("Job").agg(
    {"Salary": "avg", "Name": "count"}
).withColumnRenamed("avg(Salary)", "Avg Salary") \
 .withColumnRenamed("count(Name)", "Count") \
 .show()

+--------+----------+-----+
|     Job|Avg Salary|Count|
+--------+----------+-----+
|Engineer|   85000.0|    1|
| Manager|  120000.0|    1|
|  Intern|   45000.0|    1|
|Director|  180000.0|    1|
| Analyst|   95000.0|    1|
+--------+----------+-----+



In [5]:
# Cell 5 - Your filter + sort (this one was almost perfect!)
df_clean.filter(col("Salary") > 90000) \
        .orderBy("Salary", ascending=False) \
        .show()

+----+---+--------+------+------+------+-------+
|Name|Age|     Job|Salary|  City|Senior|  Bonus|
+----+---+--------+------+------+------+-------+
|Dana| 40|Director|180000| Plano|   Yes|18000.0|
| Bob| 35| Manager|120000|Dallas|   Yes|12000.0|
| Eve| 25| Analyst| 95000|Frisco|    No| 9500.0|
+----+---+--------+------+------+------+-------+



In [6]:
# Cell 5: Filter & sort
df_clean.filter(col("City") == 'Plano') \
        .orderBy("Salary", ascending=False) \
        .show()

+----+---+--------+------+-----+------+-------+
|Name|Age|     Job|Salary| City|Senior|  Bonus|
+----+---+--------+------+-----+------+-------+
|Dana| 40|Director|180000|Plano|   Yes|18000.0|
+----+---+--------+------+-----+------+-------+



# Step 2: Read CSV in PySpark (new cell)

In [7]:
# Read from local folder (mounted volume)
df_from_csv = spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("/home/jovyan/work/sales_data.csv")

df_from_csv.show()
df_from_csv.printSchema()   # See data types Spark guessed

+-------+----+--------+------+-------+
|   Name| Age|     Job|Salary|   City|
+-------+----+--------+------+-------+
|  Alice|  28|Engineer| 85000| Austin|
|    Bob|  35| Manager|120000| Dallas|
|Charlie|  22|  Intern| 45000|Houston|
|   Dana|  40|Director|180000|  Plano|
|    Eve|NULL| Analyst| 95000| Frisco|
|  Frank|  32|   Sales| 75000|  Plano|
+-------+----+--------+------+-------+

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- City: string (nullable = true)



# Step 3: Clean & Write as Parquet (Glue loves Parquet!)

In [9]:
# Same cleaning as before
df_clean_csv = df_from_csv \
    .withColumn("Senior", when(col("Age") >= 30, "Yes").otherwise("No")) \
    .withColumn("Bonus", col("Salary") * lit(0.10)) \
    .na.fill({"Age": 25})

# Write to Parquet (partitioned by City - common in Glue)
df_clean_csv.write.mode("overwrite") \
             .partitionBy("City") \
             .parquet("/home/jovyan/work/clean_sales_parquet")

print("Done! Check your local 'notebooks/clean_sales_parquet' folder")

Done! Check your local 'notebooks/clean_sales_parquet' folder
